In [1]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-07-24 13:15:17.222376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 13:15:17.784610: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/home2/cbisot/miniconda3/envs/amftrack/lib/python3.7/site-packages/cv2/../../lib64:
2023-07-24 13:15:17.784667: I tensorflow/stream_executor/cuda/cudart_stub.cc:29

In [1]:
plates = [
# "3_20220426",
# "12_20220502",
# "13_20220422",
# "16_20220419",
"21_20220502",
# "480_20221205",
# "28_20230227",
# "206_20230303",
# "202_20230314",
# "218_20230227",
# "219_20230307",
# "229_20230330",
# "240_20230328"
]
directory_targ = directory_project
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
# update_plate_info(directory_targ, local=True)
# all_folders = get_current_folders(directory_targ, local=True)

NameError: name 'directory_project' is not defined

In [10]:
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

select = analysis_folders

In [15]:
time = "6:40:00"
directory = directory
max_ind = 20
incr = 100

list_f = [
    get_num_trunks,
    get_area,
    get_area_separate_connected_components,
    get_num_tips,
    get_num_nodes,
    get_area_study_zone,
    get_num_tips_study_zone,
    get_num_nodes_study_zone,
    get_num_edges,
    get_length_tot,
    get_length_study_zone,
    get_is_out_study,
    get_mean_edge_straight,
    get_spore_volume,
    get_num_spores,
    get_tot_biovolume_study,
    get_tot_biovolume,
]
list_args = [{}] * len(list_f)
overwrite = True
num_parallel = 6
run_parallel_post(
    "time_plate_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    select,
    num_parallel,
    time,
    "time_plate_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3131428


In [13]:
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.functions.post_processing.time_plate import *

%load_ext autoreload
%autoreload 2
directory = directory_targ
overwrite = False
load_graphs = True
name_job = "density3"
# list_f = [
#     get_length_tot,
# ]
# list_args = [{}] * len(list_f)
max_ind = 20
incr = 100
num_parallel = 30
time = "1:00:00"
fs = [
    # get_std_tip_in_ring_new_bootstrap,
    # get_density_in_ring,
    get_density_in_ring_new,
    get_density_active_tips_in_ring,
    # get_density_in_ring_new_bootstrap,
    # get_density_anastomose_in_ring,
    # get_density_branch_rate_in_ring,
    # get_density_stop_rate_in_ring,
    # get_density_BAS_in_ring
]

list_f = []
list_args = []

for f in fs:
    list_f += [f] * 20

    list_args += [{"incr": incr, "i": i, "rh_only": True,"max_t" : 100} for i in range(0, 20)]
args = [directory, True, True]
directory_targ = directory
for index, row in select.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        whole_plate_info = whole_plate_info.loc[whole_plate_info["t"].between(0, 200)]

        run_parallel_post(
            "time_plate_post_process_long.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "density_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
            # dependency = True,
        )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

# name_job = "upload"
run_launcher(
    "analysis_uploader.py",
    [directory_targ, name_job, -1],
    plates,
    "6:00:00",
    dependency=False,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3130009


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3130010


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


...

Submitted batch job 3130249


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3130266


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3130268


In [7]:
list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    # get_width_tip_edge,
    # get_width_root_edge,
    # get_width_average,
    get_has_reached_final_pos,
    get_in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "12:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3127884


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3127885


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


...

Submitted batch job 3127944


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3127945


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3127946
